In [10]:
import numpy as np
from PIL import Image
from scipy.special import j1
from scipy.ndimage import zoom

def airy_disk(r, wavelength, hole_radius, z):
    k = 2 * np.pi / wavelength
    return (2 * j1(k * hole_radius * r / z) / (k * hole_radius * r / z)) ** 2

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, wavelength=550e-9):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image)*2000

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 对输入图像进行插值,使其尺寸与成像屏幕相匹配
    brightness_resized = zoom(brightness, magnification)

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算艾里斑图案
    airy = airy_disk(r, wavelength, hole_radius, screen_distance)

    # 将艾里斑图案与插值后的输入图像亮度进行卷积
    imaged_brightness = np.fft.ifft2(np.fft.fft2(brightness_resized) * np.fft.fft2(airy)).real

    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
output_path = 'output_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()
imaged_image.save(output_path, 'JPEG')

In [11]:
import numpy as np
from PIL import Image
from scipy.special import j1
from scipy.ndimage import zoom

def airy_disk(r, wavelength, hole_radius, z):
    k = 2 * np.pi / wavelength
    return (2 * j1(k * hole_radius * r / z) / (k * hole_radius * r / z)) ** 2

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, wavelength=550e-9):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image) / 255.0

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 对输入图像进行插值,使其尺寸与成像屏幕相匹配
    brightness_resized = zoom(brightness, magnification)

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算艾里斑图案
    airy = airy_disk(r, wavelength, hole_radius, screen_distance)

    # 将艾里斑图案与插值后的输入图像亮度进行卷积
    imaged_brightness = np.fft.fftshift(np.fft.ifft2(np.fft.fft2(brightness_resized) * np.fft.fft2(np.fft.fftshift(airy))).real)

    # 归一化亮度到 [0, 1] 范围内
    imaged_brightness = (imaged_brightness - np.min(imaged_brightness)) / (np.max(imaged_brightness) - np.min(imaged_brightness))

    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
output_path = 'output_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()
imaged_image.save(output_path, 'JPEG')

In [14]:
import numpy as np
from PIL import Image
from scipy.special import j1
from scipy.ndimage import zoom

def airy_disk(r, wavelength, hole_radius, z):
    k = 2 * np.pi / wavelength
    return (2 * j1(k * hole_radius * r / z) / (k * hole_radius * r / z)) ** 2

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, wavelength=550e-9):
    # 将图像转换为浮点数类型并归一化亮度
    brightness = np.array(image) / 255.0

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 对输入图像进行插值,使其尺寸与成像屏幕相匹配
    brightness_resized = zoom(brightness, (magnification, magnification, 1))

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算艾里斑图案
    airy = airy_disk(r, wavelength, hole_radius, screen_distance)

    # 对每个颜色通道应用卷积
    imaged_brightness = np.zeros_like(brightness_resized)
    for i in range(3):
        imaged_brightness[:, :, i] = np.fft.fftshift(np.fft.ifft2(np.fft.fft2(brightness_resized[:, :, i]) * np.fft.fft2(np.fft.fftshift(airy))).real)

    # 归一化亮度到 [0, 1] 范围内
    imaged_brightness = (imaged_brightness - np.min(imaged_brightness)) / (np.max(imaged_brightness) - np.min(imaged_brightness))

    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
output_path = 'output_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()
imaged_image.save(output_path, 'JPEG')